In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import datetime
import random

from api_keys import api_key

url = f"https://api.nal.usda.gov/ndb/reports/?"




ModuleNotFoundError: No module named 'api_keys'

In [2]:
#Creación del formulario para el usuario

user_dict = {
    "First_Name":'',
    "Last_Name": '',
    "Age":'',
    "Sex":'',
    "Weight_in_Kilograms":'',
    "Height_in_meters":''
    }



for key in user_dict:
    user_dict[key] = str(input(f"Please type your {key} "))

activitie = str(input(f"Please type your type of activitie you have: Sedentary (S), Low Active (L), Active (A) or Very Active (V) "))

if user_dict["Sex"] == "Women":
    pregnant_q = str(input(f"Please type if you are pregnant Yes(Y)/ No(N) (Only the letter) "))
else:
    pregnant_q = "N/A"

if pregnant_q == "Y":
    pregnant_a = int(input(f"In wich trimester of pregnancy are you (Only the number) ? "))
else:
    pregnant_a = "N/A"
    
if user_dict["Sex"] == "Women":
    lactation_q = str(input(f"Please type if you are in lactation Yes(Y)/ No(N) (Only the letter)"))
else:
    lactation_q = "N/A"

if lactation_q == "Y":
    lactation_a = int(input(f"Please type if you are inside the 1st or 2nd Semester of lactation (Only put the number) "))
else:
    lactation_a = "N/A"



Please type your First_Name Cristina
Please type your Last_Name Chavez
Please type your Age 24
Please type your Sex Women
Please type your Weight_in_Kilograms 61
Please type your Height_in_meters 1.67
Please type your type of activitie you have: Sedentary (S), Low Active (L), Active (A) or Very Active (V) S
Please type if you are pregnant Yes(Y)/ No(N) (Only the letter) n
Please type if you are in lactation Yes(Y)/ No(N) (Only the letter)N


In [13]:
#Cálculo del IMC actual y el ideal.

IMC_actual = round(float(user_dict["Weight_in_Kilograms"])/(float(user_dict["Height_in_meters"])**2),2)

if (IMC_actual < 24.9 and IMC_actual > 18.5):
    IMC_ideal = IMC_actual    
if IMC_actual > 24.9:
    IMC_ideal = 24.5
if IMC_actual < 18.5:
    IMC_ideal = 19
    
Ideal_Weight = round(IMC_ideal * (float(user_dict["Height_in_meters"])**2),1)

Variance_Weight = round(float(user_dict["Weight_in_Kilograms"]) - float(Ideal_Weight),1)

print(IMC_actual)
print(Ideal_Weight)
print(Variance_Weight)

21.87
61.0
0.0


In [14]:
#Cálculo de la actividad física, para la fórmula de EER

pa_data_df = pd.DataFrame({"Category": ["S", "L", "A", "V"],
                          "Men": [1,1.11,1.25,1.48],
                          "Woman": [1,1.12,1.27,1.45]})

pa_data_df = pa_data_df.set_index("Category")

if user_dict["Sex"] == "Men":
    P_Activitie = pa_data_df.loc[activitie,"Men"]
else:
    P_Activitie = pa_data_df.loc[activitie,"Woman"]

#if user_dict["Sex"] == "Men":
#    if activitie == "S":
#        P_Activitie = 1
#    elif activitie == "L":
#        P_Activitie = 1.11
#    elif activitie == "A":
#        P_Activitie = 1.25
#    elif activitie == "V":
#        P_Activitie = 1.48
#else:
#    if activitie == "S":
#        P_Activitie = 1
#    elif activitie == "L":
#        P_Activitie = 1.12
#    elif activitie == "A":
#        P_Activitie = 1.27
#    elif activitie == "V":
#        P_Activitie = 1.45


#Cálculo del requerimiento de energía diario de una persona.

EER_Men= float(662) - (9.53 * float(user_dict["Age"]))+ float(P_Activitie) *((15.91*float(Ideal_Weight)) + (539.6*float(user_dict["Height_in_meters"]))) 
EER_Women= float(354) - (6.91 * float(user_dict["Age"]))+ float(P_Activitie) *((9.36*float(Ideal_Weight)) + (float(726)*float(user_dict["Height_in_meters"])))

    
if user_dict["Sex"] == "Men":
    Nut_Req_Atnow = round(EER_Men,2)
else:
    Nut_Req_Atnow = round(EER_Women,2)
    if pregnant_q == "Y":
        if pregnant_a == 1:
            Nut_Req_Atnow = Nut_Req_Atnow
        if pregnant_a == 2:
            Nut_Req_Atnow = Nut_Req_Atnow + 340
        if pregnant_a == 3:
            Nut_Req_Atnow = Nut_Req_Atnow + 452
    if lactation_q == "Y":
        if lactation_a == 1:
            Nut_Req_Atnow = Nut_Req_Atnow + 600 - 170
        if lactation_a == 2:
            Nut_Req_Atnow = Nut_Req_Atnow + 400
            

print(f"{Nut_Req_Atnow} Kcal")


1971.54 Kcal


In [15]:
#Distribución de la energía en carbohidratos, grasas y proteina

Carb_Dist_kcal= round(Nut_Req_Atnow * .50,0)
Fat_Dist_kcal= round(Nut_Req_Atnow * .3,0)
Protein_Dist_kcal = round(Nut_Req_Atnow * .2,0)

print(Carb_Dist_kcal)
print(Fat_Dist_kcal)
print(Protein_Dist_kcal)

986.0
591.0
394.0


In [16]:
#Conversión de Kcal a gramos

Carb_Dist_gr= round(Carb_Dist_kcal / float(4),0)
Fat_Dist_gr= round(Fat_Dist_kcal / float(9),0)
Protein_Dist_gr = round(Protein_Dist_kcal / float(4),0)

print(Carb_Dist_gr)
print(Fat_Dist_gr)
print(Protein_Dist_gr)

246.0
66.0
98.0


In [17]:


User_Information=pd.DataFrame({"Name": user_dict["First_Name"],
                              "Last Name": user_dict["Last_Name"],
                               "Age": user_dict["Age"],
                               "Sex": user_dict["Sex"],
                               "Weight (Kg)": user_dict["Weight_in_Kilograms"],
                               "Height (m)": user_dict["Height_in_meters"],
                               "Pregnant": pregnant_q,
                               "Pregnancy Trimester": pregnant_a,
                               "Lactation": lactation_q,
                               "Lactation Semester": lactation_a,
                               "Actual IMC": IMC_actual,
                               "Ideal IMC": IMC_ideal,
                               "Ideal Weight(Kg)": Ideal_Weight,
                               "Extra/Missing Weight (Kg)": Variance_Weight,
                               "Physical Activitie": activitie,
                               "Energy Requirement (kcal)": Nut_Req_Atnow,
                               "Carbohydrates Req (kcal)": Carb_Dist_kcal,
                               "Fat Req (kcal)": Fat_Dist_kcal,
                               "Protein Req (kcal)": Protein_Dist_kcal,
                               "Carbohydrates Req (gr)": Carb_Dist_gr,
                               "Fat Req (gr)": Fat_Dist_gr,
                               "Protein Req (gr)": Protein_Dist_gr
                              }, index=[0])

pd.options.display.max_columns=22


User_Information

,Name,Last Name,Age,Sex,Weight (Kg),Height (m),Pregnant,Pregnancy Trimester,Lactation,Lactation Semester,Actual IMC,Ideal IMC,Ideal Weight(Kg),Extra/Missing Weight (Kg),Physical Activitie,Energy Requirement (kcal),Carbohydrates Req (kcal),Fat Req (kcal),Protein Req (kcal),Carbohydrates Req (gr),Fat Req (gr),Protein Req (gr)
0,Cristina,Chavez,24,Women,61,1.67,n,N/A,N,N/A,21.87,21.87,61.0,0.0,S,1971.54,986.0,591.0,394.0,246.0,66.0,98.0


In [2]:
# Leer el CSV

nutrients_data = 'Resources/Nutrients.csv'
nutrientsdf = pd.read_csv(nutrients_data, encoding="ISO-8859-1")
nutrientsdf.head()


,NDB_No,Nutrient_Code,Nutrient_name,Derivation_Code,Output_value,Output_uom
0,45127487,203,Protein,LCGA,4.3,g
1,45127487,204,Total lipid (fat),LCGA,18.9,g
2,45127487,205,"Carbohydrate, by difference",LCGA,72.8,g
3,45127487,207,Ash,LCGA,2.0,g
4,45127487,255,Water,LCGA,2.0,g


In [17]:
# Base de Nutrientes solo con Proteínas, Carbohidratos y Grasas

nutrients_loc=nutrientsdf.loc[nutrientsdf["Nutrient_Code"].isin([203,204,205])] 
nutrients_loc.head()

#New_Order= pd.wide_to_long()


,NDB_No,Nutrient_Code,Nutrient_name,Derivation_Code,Output_value,Output_uom
0,45127487,203,Protein,LCGA,4.3,g
1,45127487,204,Total lipid (fat),LCGA,18.9,g
2,45127487,205,"Carbohydrate, by difference",LCGA,72.8,g
29,45128706,203,Protein,LCGA,10.1,g
30,45128706,204,Total lipid (fat),LCGA,8.5,g


In [5]:
# Leer el CSV de Productos

products_data = 'Resources/Products.csv'
productsdf = pd.read_csv(products_data, encoding="ISO-8859-1")
productsdf.head()

C:\Users\maluk\Anaconda3\envs\malucastellanos\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,NDB_Number,long_name,data_source,gtin_upc,manufacturer,date_modified,date_available,ingredients_english
0,45001524,MOCHI ICE CREAM BONBONS,LI,19022128593,"G. T. Japan, Inc.",2017-11-15 19:19:38,2017-11-15 19:19:38,"ICE CREAM INGREDIENTS: MILK, CREAM, SUGAR, STR..."
1,45001528,CHIPOTLE BARBECUE SAUCE,LI,5051379043735,FRESH & EASY,2018-04-26 17:23:31,2018-04-26 17:23:31,"WATER, SUGAR, TOMATO PASTE, MOLASSES, DISTILLE..."
2,45001529,HOT & SPICY BARBECUE SAUCE,LI,5051379009434,FRESH & EASY,2018-04-26 18:17:37,2018-04-26 18:17:37,"SUGAR, WATER, DISTILLED VINEGAR, TOMATO PASTE,..."
3,45001530,BARBECUE SAUCE,LI,5051379019969,FRESH & EASY,2018-04-26 17:24:00,2018-04-26 17:24:00,"TOMATO PUREE (WATER, TOMATO PASTE), SUGAR, DIS..."
4,45001531,BARBECUE SAUCE,LI,5051379009526,FRESH & EASY,2018-04-26 17:47:41,2018-04-26 17:47:41,"SUGAR, DISTILLED VINEGAR, WATER, TOMATO PASTE,..."


In [73]:
# Obtener 25 alimentos aleatorios que se consumen en un día

selected_products = productsdf.sample(n=25)
selected_products


,NDB_Number,long_name,data_source,gtin_upc,manufacturer,date_modified,date_available,ingredients_english
183767,45291079,CARIBBEAN SPICE BARBECUE SAUCE,LI,011110025968,The Kroger Co.,2017-07-14 19:24:39,2017-07-14 19:24:39,"TOMATO PUREE (WATER, TOMATO PASTE), HIGH FRUCT..."
197031,45325685,OVEN BAKED SUNDAY MEATBALLS,LI,026825003613,John Zidian Company Inc.,2017-08-11 17:32:22,2017-08-11 17:32:22,"BEEF, WATER, TEXTURED SOY PROTEIN CONCENTRATE ..."
94471,45181805,ORGANIC ROMAINE HEARTS,LI,3003430300,"Giant Eagle, Inc.",2017-07-14 18:25:59,2017-07-14 18:25:59,ORGANIC ROMANE LETTUCE.
199599,45329172,DEEP DISH PIZZA,LI,829006000100,THE DISH FAMOUS PIZZA,2017-11-16 16:22:19,2017-11-16 16:22:19,"CRUST: ENRICHED WHEAT FLOUR, MALTED BARLEY, NI..."
185216,45292722,"QUAKER, REAL MEDLEYS, GRANOLA & YOGURT BLEND, ...",LI,030000321492,The Quaker Oats Company,2017-10-13 06:34:29,2017-10-13 06:34:29,"MULTIGRAIN GRANOLA (WHOLE GRAIN ROLLED OATS, W..."
8443,45018290,"ASSI, ORIENTAL STYLE NOODLE (PASTA)",LI,81652081135,Rhee Bros. Inc.,2017-07-14 19:30:32,2017-07-14 19:30:32,"HEAT FLOUR SALT, WATER"
58182,45136517,HANDCRAFTED FROZEN POPS,LI,856767003255,Luna Pops LLC,2018-01-20 17:33:58,2018-01-20 17:33:58,"CREAM BASE (CREAM, MILK, CORN SYRUP, SUGAR, SW..."
107924,45197183,PRETZEL ROLLS,LI,71314060059,"Perfection Bakeries, Inc.",2017-12-30 05:16:02,2017-12-30 05:16:02,"ENRICHED WHEAT FLOUR, (WHEAT FLOUR, MALTED BAR..."
130439,45222215,CREAMY BROCCOLI CHEDDAR CHEESE SOUP,LI,855407004058,STORE HOUSE FOODS,2017-07-14 17:07:01,2017-07-14 17:07:01,"POTATO FLAKES, SHARP CHEDDAR CHEESE POWDER (PA..."
191264,45315633,PEACH HALVES IN HEAVY SYRUP,LI,04656701587,Raley's,2018-04-05 15:46:22,2018-04-05 15:46:22,"PEACHES, WATER, CORN SYRUP, HIGH FRUCTOSE CORN..."


In [74]:
# Obtener las proteínas, carbohidratos y grasas de los alimentos seleccionados

products_nutrients = selected_products.loc[:,["NDB_Number", "long_name"]]
products_nutrients


,NDB_Number,long_name
183767,45291079,CARIBBEAN SPICE BARBECUE SAUCE
197031,45325685,OVEN BAKED SUNDAY MEATBALLS
94471,45181805,ORGANIC ROMAINE HEARTS
199599,45329172,DEEP DISH PIZZA
185216,45292722,"QUAKER, REAL MEDLEYS, GRANOLA & YOGURT BLEND, ..."
8443,45018290,"ASSI, ORIENTAL STYLE NOODLE (PASTA)"
58182,45136517,HANDCRAFTED FROZEN POPS
107924,45197183,PRETZEL ROLLS
130439,45222215,CREAMY BROCCOLI CHEDDAR CHEESE SOUP
191264,45315633,PEACH HALVES IN HEAVY SYRUP


In [85]:
products_index = products_nutrients["NDB_Number"].unique()
products_index
        

array([45291079, 45325685, 45181805, 45329172, 45292722, 45018290,
       45136517, 45197183, 45222215, 45315633, 45182778, 45238060,
       45351616, 45019265, 45048038, 45139486, 45239182, 45121117,
       45234306, 45237477, 45159123, 45348749, 45041920, 45054020,
       45040058], dtype=int64)

KeyboardInterrupt: 

In [86]:
# Base de nutrientes solo con la muestra aleatoria

nutrients_sample=nutrients_loc.loc[nutrients_loc["NDB_No"].isin(products_index)]
nutrients_sample

,NDB_No,Nutrient_Code,Nutrient_name,Derivation_Code,Output_value,Output_uom
80295,45121117,203,Protein,LCCS,9.35,g
80296,45121117,204,Total lipid (fat),LCCS,11.51,g
80297,45121117,205,"Carbohydrate, by difference",LCCS,28.78,g
211270,45139486,203,Protein,LCCS,19.64,g
211271,45139486,204,Total lipid (fat),LCCS,13.39,g
211272,45139486,205,"Carbohydrate, by difference",LCCS,3.57,g
225144,45238060,203,Protein,LCCS,0.00,g
225145,45238060,204,Total lipid (fat),LCCD,0.00,g
225146,45238060,205,"Carbohydrate, by difference",LCCS,9.17,g
324291,45181805,203,Protein,LCCS,1.18,g


In [87]:
# Obtener DataFrames solo con Proteínas, Grasas y Carbohidratos y luego combinarlo todo en un nuevo Data Frame

nutrients_proteins = nutrients_sample.loc[nutrients_sample["Nutrient_Code"] == 203]
proteins = nutrients_proteins.loc[:,["NDB_No", "Output_value"]]
proteins = proteins.rename(columns = {"NDB_No": "NDB_Number"})
proteins



,NDB_Number,Output_value
80295,45121117,9.35
211270,45139486,19.64
225144,45238060,0.00
324291,45181805,1.18
418644,45040058,2.35
508656,45351616,0.51
770489,45041920,4.11
975781,45159123,20.00
997873,45292722,9.23
1081529,45348749,12.50


In [88]:
nutrients_fats = nutrients_sample.loc[nutrients_sample["Nutrient_Code"] == 204]
fats = nutrients_fats.loc[:,["NDB_No", "Output_value"]]
fats = fats.rename(columns = {"NDB_No": "NDB_Number"})
fats




,NDB_Number,Output_value
80296,45121117,11.51
211271,45139486,13.39
225145,45238060,0.00
324292,45181805,0.00
418645,45040058,0.00
508657,45351616,0.00
770490,45041920,16.44
975782,45159123,50.00
997874,45292722,3.85
1081530,45348749,6.25


In [89]:
nutrients_carbs = nutrients_sample.loc[nutrients_sample["Nutrient_Code"] == 205]
carbs = nutrients_carbs.loc[:,["NDB_No", "Output_value"]]
carbs = carbs.rename(columns = {"NDB_No": "NDB_Number"})
carbs


,NDB_Number,Output_value
80297,45121117,28.78
211272,45139486,3.57
225146,45238060,9.17
324293,45181805,3.53
418646,45040058,14.12
508658,45351616,13.13
770491,45041920,56.16
975783,45159123,23.33
997875,45292722,80.00
1081531,45348749,70.00


In [103]:
# Merge all the data to have one dataframe with all the info

products_nutrients = pd.merge(products_nutrients, proteins, on = "NDB_Number")
products_nutrients = products_nutrients.rename(columns = {"Output_value": "Proteins (gr)"})
products_nutrients = pd.merge(products_nutrients, fats, on = "NDB_Number")
products_nutrients = products_nutrients.rename(columns = {"Output_value": "Total Lipid (fat) (gr)"})
products_nutrients = pd.merge(products_nutrients, carbs, on = "NDB_Number")
products_nutrients = products_nutrients.rename(columns = {"Output_value": "Carbohydrate (gr)"})
products_nutrients


,NDB_Number,long_name,Proteins (gr),Total Lipid (fat) (gr),Carbohydrate (gr)
0,45291079,CARIBBEAN SPICE BARBECUE SAUCE,0.00,0.00,27.78
1,45325685,OVEN BAKED SUNDAY MEATBALLS,15.29,21.18,5.88
2,45181805,ORGANIC ROMAINE HEARTS,1.18,0.00,3.53
3,45329172,DEEP DISH PIZZA,10.07,14.39,28.06
4,45292722,"QUAKER, REAL MEDLEYS, GRANOLA & YOGURT BLEND, ...",9.23,3.85,80.00
5,45018290,"ASSI, ORIENTAL STYLE NOODLE (PASTA)",10.71,3.57,71.43
6,45136517,HANDCRAFTED FROZEN POPS,2.22,6.67,32.22
7,45197183,PRETZEL ROLLS,7.50,3.75,47.50
8,45222215,CREAMY BROCCOLI CHEDDAR CHEESE SOUP,18.60,13.95,48.84
9,45315633,PEACH HALVES IN HEAVY SYRUP,0.78,0.00,19.53


In [104]:
# Consumo total de proteína, grasa y carbohidratos
total_protein = sum(products_nutrients["Proteins (gr)"])
total_fats = sum(products_nutrients["Total Lipid (fat) (gr)"])
total_carbs = sum(products_nutrients["Carbohydrate (gr)"])

print("Your consume of proteins of today is: " + str(round(total_protein,2)) + " gr")
print("Your consume of fats of today is: " + str(round(total_fats,2)) + " gr")
print("Your consume of carbs of today is: " + str(round(total_carbs,2)) + " gr")
 

Your consume of proteins of today is: 185.18 gr
Your consume of fats of today is: 254.59 gr
Your consume of carbs of today is: 755.02 gr
